# Single shortest path

Single shortest path algorithm implemented by QUBO in quantum annealer

In [59]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [60]:
edge = np.array([(0, 1, 5), (0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4)])
vertices = 4
start = 0
end = 3

Above graph visualised:

![](graph1.png)

In [61]:
#edge = np.array([(0, 1, 8), (1, 3, 5), (0, 2, 2), (2, 1, 4), (2, 4, 7), (4, 3, 3)])
#vertices = 5
#start = 0
#end = 3

Above graph visualised:
    
![](graph2.png)

In [63]:
#edge = np.array([(0, 1, 5), (0, 2, 5), (1, 0, 5), (1, 2, 2), (1, 3, 2),
#                 (2, 0, 5), (2, 1, 2), (2, 3, 10), (3, 1, 2), (3, 2, 10)])
#vertices = 4
#start = 0
#end = 3

Above graph visualised (every edge is biderctional):
    
![](graph3.png)

## Construct max_count, edge_names and edge_weights variables

In [64]:
max_count = 0
edges = len(edge)
edge_names = {}
edge_weights = {}
for i,e in enumerate(edge):
    max_count += e[2]
    edge_names[i] = str(e[0]) + '-' + str(e[1])
    edge_weights[edge_names[i]] = e[2]
print('Max count:',max_count)

def result_info(sampleset):
    r = sampleset.first.sample
    r = dict((k, edge_weights[k]) for k, v in r.items() if v==1)
    print('Edges: ' + str(r) + '\nTotal weights:', str(sum(r.values())))

Max count: 15


## Create QUBO from edge array

Based on "Directed Edge-Based Approach" from Kraus & McCollum (2020) Solving the Network Shortest Path Problem on a Quantum Annealer.

https://doi.org/10.1109/TQE.2020.3021921


Constraints: 

- the source vertex has one more edge leaving it than entering it
- the terminal vertex has one more edge entering it than leaving it
- every other vertex has as many edges entering it as it has leaving it.



In [71]:
Q = np.zeros((edges,edges))

t1 = time.time()

# Include weights of edges

for i,e in enumerate(edge):
    Q[i][i] = e[2]        

# set constraints for starting node, ending node and other nodes for incoming and outgoing edges

for i,e in enumerate(edge):
    Q[i][i] += max_count * 2               # Every edge is incomig and outgoing to a vertex


for x in range(vertices):
    ec = np.zeros(edges)                   # Mark edges connected to vertex x, input (+1) or output (-1). 
    for i,e in enumerate(edge):
        if e[0]==x:
            ec[i] = 1
        if e[1]==x:
            ec[i] = -1

    for i in range(edges):                     # Loop through all edge combinations connected to vertex x, and 
        for j in range(i+1, edges):            #   add penalty to edge pair if both are incoming or outgoing to   
            Q[i][j] += ec[i]*ec[j]*2*max_count #   vertex x, and negative penalty otherwise.

    for i in range(edges):
        if x==end:                         # If we are in ending vertex, add penalty if vertex is input of edge i, 
            Q[i][i] += ec[i]*2*max_count   #   and negative penalty if vertex is output.
        if x==start:                       # Similar mirrored operation if we are in starting vertex. 
            Q[i][i] -= ec[i]*2*max_count

t2 = time.time()
print('Time used for construction Q (ms): {:.3f}\n'.format((t2-t1)*1000))
print(edge_names)
print(Q)
print(sum(sum(Q)))

Time used for construction Q (ms): 0.498

{0: '0-1', 1: '0-2', 2: '2-1', 3: '1-3', 4: '3-2'}
[[  5.  30.  30. -30.   0.]
 [  0.   1. -30.   0.  30.]
 [  0.   0.  32. -30. -30.]
 [  0.   0.   0.   3. -30.]
 [  0.   0.   0.   0.  64.]]
45.0


## Create BQM from QUBO

In [72]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm = bqm.relabel_variables(edge_names, inplace=False)

## Local deterministic classical solver

In [73]:
sampleset = dimod.ExactSolver().sample(bqm)
print(sampleset.truncate(10))

  0-1 0-2 1-3 2-1 3-2 energy num_oc.
0   0   1   1   1   0  -24.0       1
1   1   0   1   0   0  -22.0       1
2   0   0   0   0   0    0.0       1
3   0   1   0   0   0    1.0       1
4   0   0   1   0   0    3.0       1
5   0   1   0   1   0    3.0       1
6   0   1   1   0   0    4.0       1
7   1   0   0   0   0    5.0       1
8   0   0   1   1   0    5.0       1
9   1   1   1   0   0    9.0       1
['BINARY', 10 rows, 10 samples, 5 variables]


In [68]:
result_info(sampleset)

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6


## Local heuristic classical solver

In [69]:
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500)
print(sampleset2.aggregate())

  0-1 0-2 1-3 2-1 3-2 energy num_oc.
0   0   1   1   1   0  -24.0     291
1   1   0   1   0   0  -22.0     209
['BINARY', 2 rows, 500 samples, 5 variables]


In [70]:
result_info(sampleset2)

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6


## Quantum solver

In [12]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [13]:
sampleset3 = EmbeddingComposite(machine).sample(bqm, num_reads=500)
print(sampleset3) 

  0-1 0-2 1-3 2-1 3-2 energy num_oc. chain_.
0   0   1   1   1   0  -24.0     247     0.0
1   1   0   1   0   0  -22.0     250     0.0
2   0   0   0   0   0    0.0       1     0.0
3   0   0   1   1   0    5.0       1     0.0
4   1   0   1   1   0   10.0       1     0.2
['BINARY', 5 rows, 500 samples, 5 variables]


In [14]:
time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(time))
print('Physical qubits used: {}'.format(qubits))

QPU time used (ms): 68.3
Physical qubits used: 6


In [15]:
result_info(sampleset3)

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6


In [16]:
dwave.inspector.show(sampleset3)

Serving Inspector on http://127.0.0.1:18000/?problemId=23aecf5e-de20-44c6-90d2-3b9dc4faaf06

'http://127.0.0.1:18000/?problemId=23aecf5e-de20-44c6-90d2-3b9dc4faaf06'

## Hybrid solver

In [17]:
sampleset4 = LeapHybridSampler().sample(bqm)
print(sampleset4) 

  0-1 0-2 1-3 2-1 3-2 energy num_oc.
0   0   1   1   1   0  -24.0       1
['BINARY', 1 rows, 1 samples, 5 variables]


In [18]:
time = sampleset4.info['qpu_access_time'] / 1000
print('QPU time used (ms): {:.1f}'.format(time))

QPU time used (ms): 96.4


In [19]:
result_info(sampleset4)

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6
